In [1]:
import requests
import datetime
import json
import pandas as pd

In [2]:
# 1. What is the root URL?
ROOT_URL = "https://api.binance.com/"

In [3]:
#2. What is the endpoint to retrieve klines (open-high-low-close data) for a specific cryptocurrency?
endpoint= "api/v3/klines"

In [4]:
#convert date to unix time
from datetime import timezone

date = datetime.datetime(2022, 9, 1)
unix_ts_utc = date.replace(tzinfo=timezone.utc).timestamp()
unix_ts = int(date.timestamp())
# Binace requires dates in miliseconds, so multiply by 1000
unix_ts = unix_ts*1000
print(unix_ts)

1661983200000


In [5]:
# 3. Specify a request string to retrieve 75 observations of klines data for BTCUSDT since 2022-09-01.
# construct request URL
req = "{root_url}{endpoint}?symbol={symbol}USDT&limit={limit}&interval=1h&startTime={since}" \
     .format(root_url=ROOT_URL,
             endpoint=endpoint,
             limit = 75,
             symbol="BTC",
             since=unix_ts)
print(req)

https://api.binance.com/api/v3/klines?symbol=BTCUSDT&limit=75&interval=1h&startTime=1661983200000


In [6]:
# send request to the Binance server & receive a response
resp = requests.get(req)
print(resp)

<Response [200]>


In [7]:
# convert to DataFrame
# to do's: convert unix time variables to datetime variables
# weirdly enough the response throws back klines from 2017, why is that? dates seems to be correctly specified above
# EDIT: Binance requires milisecons for the timestamp, so unix_ts above is multiplied by 1000 to solve the problem
data = pd.DataFrame.from_records(
    resp.json(),
    columns=["kline_open_time", "open_price", "high_price", "low_price", "close_price", "volume", "kline_close_time", "quote_asset_volume", "number_of_trades", "Taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "unused_field_ignore"]

)
# Convert unix timestamps to human-readable dates
data["kline_open_time"] = pd.to_datetime(data.kline_open_time, unit="ms", origin="unix")
data["kline_close_time"] = pd.to_datetime(data.kline_close_time, unit="ms", origin="unix")

print(data.head())
print(data.shape)

      kline_open_time      open_price      high_price       low_price  \
0 2022-08-31 22:00:00  20280.54000000  20292.75000000  20085.70000000   
1 2022-08-31 23:00:00  20155.00000000  20157.53000000  19966.00000000   
2 2022-09-01 00:00:00  20048.44000000  20138.15000000  19917.01000000   
3 2022-09-01 01:00:00  20064.43000000  20185.09000000  20051.22000000   
4 2022-09-01 02:00:00  20127.45000000  20208.37000000  20069.35000000   

      close_price          volume        kline_close_time  quote_asset_volume  \
0  20155.00000000   9926.63922000 2022-08-31 22:59:59.999  200115143.50616790   
1  20050.02000000   8540.54148000 2022-08-31 23:59:59.999  171113575.27829430   
2  20064.43000000  11108.70225000 2022-09-01 00:59:59.999  222708634.02234470   
3  20127.45000000   7942.65870000 2022-09-01 01:59:59.999  159810609.65936390   
4  20118.09000000   7256.77429000 2022-09-01 02:59:59.999  146240287.43752270   

   number_of_trades Taker_buy_base_asset_volume taker_buy_quote_asset_volu

In [8]:
# 4. Write a function (in Python, R or Julia) that retrieves 75 observations of klines data for a generic
# currency pair since a generic date.The function should take the currency pair and start date as input parameters.
def get_klines_data_binance(startTime, symbol="BTCUSDT", invertal="1d", limit=75, to_csv=False):
    """
    Function to get klines data from binance for a specified symbol. 
    Parameters:
    symbol is the ID of the pair of crypto/currency desired, default us BTCUSDT
    interval is the periodicity of the wanted data, default is daily
    startTime is the date from which we want our data to start, sould be a unix timestamp in miliseconds
    limit is the number of observations we want to get
    to_csv logical to know if data should be written into a csv, default is False
    """
    # Verify that startTime is given
    try:
        assert isinstance(startTime, (str, int)), "startTime must be an integer or a string"
    except:
        print("You must specify a startTime")
    
    # Define the query
    req = "{root_url}{endpoint}?symbol={symbol}&limit={limit}&interval=1h&startTime={since}" \
     .format(root_url='https://api.binance.com/',
             endpoint='api/v3/klines',
             limit = limit,
             symbol=symbol,
             since=startTime)
    # Call the API
    resp = requests.get(req)
    # Check if the query was succesful
    assert resp.status_code == 200, "Error in the query, check parameters"
    
    # Finally, parse retrieved data into a data frame.
    data = pd.DataFrame.from_records(
        resp.json(),
        columns=["kline_open_time", "open_price",
                 "high_price", "low_price", "close_price",
                 "volume", "kline_close_time",
                 "quote_asset_volume", "number_of_trades",
                 "Taker_buy_base_asset_volume",
                 "taker_buy_quote_asset_volume",
                 "unused_field_ignore"]
    )
    # Drop unused field
    data.drop("unused_field_ignore", axis=1, inplace=True)
    # Convert unix timestamps to human-readable dates
    data["kline_open_time"] = pd.to_datetime(data.kline_open_time, unit="ms", origin="unix")
    data["kline_close_time"] = pd.to_datetime(data.kline_close_time, unit="ms", origin="unix")
    
    if to_csv:
        data.to_csv("query_result.csv")
    
    return data

Test the function:

In [9]:
data_btcusd = get_klines_data_binance(startTime=unix_ts, symbol="BTCUSDT")
print(data_btcusd.head())
print(data_btcusd.shape)

      kline_open_time      open_price      high_price       low_price  \
0 2022-08-31 22:00:00  20280.54000000  20292.75000000  20085.70000000   
1 2022-08-31 23:00:00  20155.00000000  20157.53000000  19966.00000000   
2 2022-09-01 00:00:00  20048.44000000  20138.15000000  19917.01000000   
3 2022-09-01 01:00:00  20064.43000000  20185.09000000  20051.22000000   
4 2022-09-01 02:00:00  20127.45000000  20208.37000000  20069.35000000   

      close_price          volume        kline_close_time  quote_asset_volume  \
0  20155.00000000   9926.63922000 2022-08-31 22:59:59.999  200115143.50616790   
1  20050.02000000   8540.54148000 2022-08-31 23:59:59.999  171113575.27829430   
2  20064.43000000  11108.70225000 2022-09-01 00:59:59.999  222708634.02234470   
3  20127.45000000   7942.65870000 2022-09-01 01:59:59.999  159810609.65936390   
4  20118.09000000   7256.77429000 2022-09-01 02:59:59.999  146240287.43752270   

   number_of_trades Taker_buy_base_asset_volume taker_buy_quote_asset_volu